In [104]:
import numpy as np
eats = np.load('./rest_name_snuload.npy', allow_pickle=True)
len(eats)

230

In [18]:
df_1 = pd.read_csv('./data/샤로수길사업장.csv', sep=',', encoding='cp949')
df_1

,Unnamed: 0,검색명,사업장명,인허가일자,도로명주소
0,204588,청신호,청신호,20190108,관악로12
1,204593,데일리오아시스,데일리오아시스 샤로수길점,20190326,관악로14
2,204596,하노이별,하노이별 서울대입구점,20190228,관악로12
3,204602,까까,까까,20190219,관악로12
4,204615,하라고지페,하라고지페(haragogipe),20190227,관악로14
...,...,...,...,...,...
254,474932,가득,가득 한끼,20120503,남부순환로230
255,475052,포케올데이,포케올데이 서울대입구점,20171019,관악로12
256,475118,슬라드,슬라드 후카,20220509,관악로16
257,475538,복희,복희,20210517,관악로14


In [38]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
from tqdm import tqdm

street_name='인사동'
df_original=pd.read_csv('./data/'+street_name+'사업장.csv',sep=',', encoding='cp949')
n_stores=len(df_original)

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
URL = 'https://map.naver.com/v5/?c=14122775.1818058,4513294.7120075,15,0,0,0,dh'
browser.get(URL)

c = ['음식점 이름','인허가일자','주소', '분류','평점', '방문자 리뷰 수', '블로그 리뷰 수', '메뉴']
# df = pd.DataFrame(columns = c)
n = 30
m = 5

for i in tqdm(range(224, n_stores)):
    real_name=df_original.loc[i]['사업장명']
    name=df_original.loc[i]['검색명']
    loc=df_original.loc[i]['도로명주소']
    start_date=df_original.loc[i]['인허가일자']
    
    menu_list = []
    menu_text = ""
    #review_list = []
    
    search_box = WebDriverWait(browser, n).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'input_search'))
    )
    #search_box = browser.find_element(By.CLASS_NAME, 'input_search')
    search_box.send_keys(name)
    
    try:
        time.sleep(2)
        pages = WebDriverWait(browser, m).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list'))
        )
        #pages = browser.find_elements(By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list')
        if len(pages) == 1:
            address_list = WebDriverWait(browser, m).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list > ul > li > a > div > div.place_text_box.sub.ng-star-inserted > span'))
            )
            for idx, ad in zip(range(len(address_list)), address_list):
                '''
                    주소 확인 (주소는 ad.text로 확인)
                ''' 
                if ad.text.find(loc) != -1:
                    WebDriverWait(browser, m).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, f'#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list > ul > li:nth-child({idx+1}) > a > div > div:nth-child(1) > span.place_text'))
                    ).click()
                    break
            #browser.find_element(By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list > ul > li:nth-child(1) > a > div > div:nth-child(1) > span.place_text').click()
        elif len(pages) == 2:
            address_list = WebDriverWait(browser, m).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container:nth-child(2) > div > nm-list > ul > li > a > div > div.place_text_box.sub.ng-star-inserted > span'))
            )
            for idx, ad in zip(range(len(address_list)), address_list):
                '''
                    위의 주소 확인 코드랑 동일하게 작성
                '''
                if ad.text.find(loc) != -1:
                    WebDriverWait(browser, m).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, f'#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container:nth-child(2) > div > nm-list > ul > li:nth-child({idx+1}) > a > div > div:nth-child(1) > span.place_text'))
                    ).click()
                    break
            #browser.find_element(By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container:nth-child(2) > div > nm-list > ul > li:nth-child(1) > a > div > div:nth-child(1) > span.place_text').click()
    
        #time.sleep(2)
        
        browser.switch_to.default_content()
        frame = WebDriverWait(browser, m).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#entryIframe'))
                )
        #frame = browser.find_element(By.CSS_SELECTOR, "#entryIframe")
        browser.switch_to.frame(frame)
        
    except Exception as e:
        rate = 0
        num_of_visitor_reviews = 0
        num_of_blog_reviews = 0
        category = '카테고리가 없습니다'
        address = '주소가 없습니다'
        
    else:
        # 평점, 리뷰 개수 크롤링
        time.sleep(2)
        try:
            rate = WebDriverWait(browser, m).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span._1Y6hi._1A8_M > em'))
                    ).text
            
            #rate = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div._37n49 > span._1Y6hi._1A8_M > em').text
        except:
            rate = '평점이 없습니다'
            try:
                num_of_visitor_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(1) > a > em'))
                                        ).text
                #num_of_visitor_reviews = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div._37n49 > span:nth-child(1) > a > em').text
            except:
                num_of_visitor_reviews = '방문자 리뷰가 없습니다'
            
            try:
                num_of_blog_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(2) > a > em'))
                                        ).text
                #num_of_blog_reviews = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div._37n49 > span:nth-child(2) > a > em').text
            except:
                num_of_blog_reviews = '블로그 리뷰가 없습니다'
        
        else:
            try:
                num_of_visitor_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(2) > a > em'))
                                        ).text
                                                                                            
                #num_of_visitor_reviews = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div._37n49 > span:nth-child(2) > a > em').text
            except:
                num_of_visitor_reviews = '방문자 리뷰가 없습니다'
            
            try:
                num_of_blog_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(3) > a > em'))
                                        ).text
                #num_of_blog_reviews = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div._37n49 > span:nth-child(3) > a > em').text
            except:
                num_of_blog_reviews = '블로그 리뷰가 없습니다'
            
        # 카테고리 크롤링    
        category = WebDriverWait(browser, m).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#_title > span._3ocDE'))
        ).text
        #category = browser.find_element(By.CSS_SELECTOR, '#_title > span._3ocDE').text
        
        # 주소 크롤링
        address = WebDriverWait(browser, m).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin._18vYz > div > ul > li._1M_Iz._1aj6- > div > a > span._2yqUQ'))
        ).text
        #address = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin._18vYz > div > ul > li._1M_Iz._1aj6- > div > a > span._2yqUQ').text
        
        # 메뉴 및 리뷰 크롤링
        titles = WebDriverWait(browser, m).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a > span'))
        )
        #titles = browser.find_elements(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a > span')
        for title in titles:
            if title.text == '메뉴':
                # 메뉴 크롤링
                title.click()
                try:
                    menu_names = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin > div > ul > li > a > div._2CZ7z > div._25ryC > div > span._3yfZ1'))
                                )
                    menu_prices = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin > div > ul > li > a > div._2CZ7z > div._3qFuX'))
                                )
                except:
                    try:
                        menu_names = WebDriverWait(browser, m).until(
                                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#root > div.naver_order_contents > div > div > div.order_list > div.order_list_wrap.store_delivery > div > ul > li > div > a > div > div.tit'))
                                    )
                        menu_prices = WebDriverWait(browser, m).until(
                                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#root > div.naver_order_contents > div > div > div.order_list > div.order_list_wrap.store_delivery > div > ul > li > div > a > div > div.price'))
                                    )
                    except:
                        menu_names = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#content > div.place_tab_detail > div.sc_box.d_main_menu_list > ul > li > a.list_link > div.info_area > div.tit.ellp > span > div'))
                                )
                        menu_prices = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#content > div.place_tab_detail > div.sc_box.d_main_menu_list > ul > li > a.list_link > div.info_area > div.price'))
                                )
                #menu_names = browser.find_elements(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin > div > ul > li > a > div._2CZ7z > div._25ryC > div > span')
                #menu_prices = browser.find_elements(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin > div > ul > li > a > div._2CZ7z > div._3qFuX')
                for menu_name, menu_price in zip(menu_names, menu_prices):
                    mpt=menu_price.text.replace(",","")[:-1]
                    menu_list.append(menu_name.text+':'+mpt)
                    menu_text=",".join(menu_list)
                  
                break
            '''
            elif title.text == '리뷰':
                # 리뷰 크롤링
                title.click()
                time.sleep(5)
                reviews = browser.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[3]/div[1]/div/ul/li/div[2]/a/span')
                #r = browser.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.cXO6M > div.place_section_content > div > ul > li:nth-child(1) > div._3vfQ6 > a > span')
                #print(r.text)
                for review in reviews[:5]:
                    review_list.append(review.text)
            '''
        
        df = df.append(dict(zip(c, [real_name,start_date,address,category, rate, num_of_visitor_reviews, num_of_blog_reviews,  menu_text])), ignore_index=True)

        
        
    finally:
        time.sleep(1)
        browser.switch_to.default_content()
        WebDriverWait(browser, m).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, '#sidebar > navbar > perfect-scrollbar > div > div.ps-content > div > ul > li:nth-child(1) > a'))
                            ).click()
        #browser.find_element(By.CSS_SELECTOR, '#sidebar > navbar > perfect-scrollbar > div > div.ps-content > div > ul > li:nth-child(1) > a').click()
        print(real_name,start_date,address,category, rate, num_of_visitor_reviews, num_of_blog_reviews,  menu_text)




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\thsth\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
  0%|▎                                                                                 | 1/310 [00:07<37:33,  7.29s/it]

양지식당 별관 20160408 서울 종로구 율곡로29길 4 백반,가정식 4.47 211 96 백반:6000


  1%|▌                                                                                 | 2/310 [00:13<35:14,  6.86s/it]

우르르 20160408 서울 종로구 율곡로19길 41-12 2층 우르르 카페,디저트 4.9 58 24 카페라떼:4500,펌킨 파이:5500,청귤에이드:6000,애플 캬라멜 티라미수:6000,치즈 케이크:5000,홀케이크 (예약 필수):변


  1%|▊                                                                                 | 3/310 [00:20<34:02,  6.65s/it]

...

음식당 19800809 서울 종로구 인사동길 37-3 1층 음식당 카페,디저트 4.54 82 46 아이스크림 크로플:6000,마약로제떡볶이:7500,마약떡볶이:5500,딸기라떼:6000,아메리카노:3800,크로플:4500,갈릭크로플:5500,체다치즈 크로플:5500,아메리카노 테이크아웃:2000


100%|█████████████████████████████████████████████████████████████████████████████▋| 309/310 [1:19:19<00:09,  9.31s/it]

뮤직컴플렉스서울 20220512 서울 종로구 인사동길 49 5층 카페,디저트 평점이 없습니다 5 5 에티오피아 타데:10000,에티오피아 게이샤:10000,에티오피아 트와콕:10000,스페셜티 커피:10000,코스타리카 알뚜라:14000


100%|██████████████████████████████████████████████████████████████████████████████| 310/310 [1:19:28<00:00, 15.38s/it]

아라가야 20211006 주소가 없습니다 카테고리가 없습니다 0 0 0 


DataFrame csv 파일로 저장

In [35]:
df.to_csv('./data/'+street_name+'크롤링결과.csv')

혹시 모를 상황을 대비하여 각각의 크롤링 df를 변수로 저장함

In [24]:
snu_road_df = df

In [25]:
# 259개 중 231개 크롤링 됨.
snu_road_df

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,청신호,20190108,서울 관악구 관악로12길 30,"카페,디저트",4.55,80,100,"바스크치즈케이크:5500,크로플:9000,아메리카노:4500,바닐라 라떼:5500,..."
1,데일리오아시스 샤로수길점,20190326,서울 관악구 관악로14길 74-3 1층,"카페,디저트",평점이 없습니다,550,464,"데일리오아시스:7000,말차먹었소:7000,플레인 티라미수:8000"
2,하노이별 서울대입구점,20190228,서울 관악구 관악로12길 3-14 1층,베트남음식,4.51,320,68,"어린이 양지쌀국수:5500,홍두깨:8500,양지차돌:8000,우목심:8500,콤보(..."
3,까까,20190219,서울 관악구 관악로12길 38 1층,"카페,디저트",4.61,128,177,"아메리카노:4000,라떼:4500,더블치즈케이크:6500,딸기타르트:6000,딸기생..."
4,하라고지페(haragogipe),20190227,서울 관악구 관악로14길 106 지하 1층,와인,4.48,113,191,"치즈 플레이트:14000,와인보틀:변,카프레제:9000"
...,...,...,...,...,...,...,...,...
226,더멜팅팟,20151008,서울 관악구 관악로14길 106 1층 (102호),햄버거,4.44,319,660,"아보카도 버거:14000,디트로이트 더블 버거:14000,클래식 치즈 버거:1000..."
227,부엌우동집,20210504,서울 관악구 남부순환로226길 23-7 1층,"우동,소바",4.84,187,36,"순한우동:8000,얼큰우동:8000,냉모밀(여름한정):8000,유부초밥 (2p):1..."
228,가득 한끼,20120503,서울 관악구 남부순환로230길 39 1층,한식,평점이 없습니다,1,블로그 리뷰가 없습니다,"우삼겹 가득비빔밥:9900,육전 가득 비빔밥:10900,항정살 가득 비빔밥:9900..."
229,슬라드 후카,20220509,서울 관악구 관악로16길 46 지하 1층,바(BAR),평점이 없습니다,1,블로그 리뷰가 없습니다,입력예정:1


In [28]:
dkb_market_df = df

In [29]:
# 20개 중 17개 크롤링 됨.
dkb_market_df

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,오대양횟집,19950814,서울 노원구 동일로180길 37 공릉상가,생선회,4.24,68,3,
1,하랑이네식당,20110502,서울 노원구 동일로180길 41 녹산빌딩,한식,평점이 없습니다,2,블로그 리뷰가 없습니다,"백반:6000,닭도리탕(2인이상):30000,제육볶음(2인이상):12000,동태찌게..."
2,불사조,20110826,서울 노원구 동일로180길 41 녹산빌딩 1층,덮밥,4.42,46,22,"(직화)제육덮밥:8000,(직화)한근 제육:15000"
3,공릉카페,20130121,서울 노원구 동일로180길 61-7,"카페,디저트",4.49,464,188,"드립커피:5000,에스프레소:4000,아메리카노:4000,플랫화이트:4500,카페라..."
4,도깨비정육식당,20130705,서울 노원구 동일로180길 48-9 광덕아파트제상가동제1층 101,"육류,고기요리",4.33,145,2,"호주산 등심불고기 (600g):12000,호주산 양지머리 (600g):14000,수..."
5,카페루바토,20140424,서울 노원구 동일로180길 53 1층 키페루바토,"카페,디저트",4.33,127,블로그 리뷰가 없습니다,"아메리카노(HOT):2000,카페라테(HOT):2500,아메리카노(ICE):2500..."
6,소문난순대족발,20111209,서울 노원구 동일로180길 53 본향성결교회,"순대,순댓국",4.28,287,18,ㅋ:2222
7,가마솥 왕족발,20150820,서울 노원구 동일로180길 25,"족발,보쌈",4.39,393,18,"가마솥 왕족발(특대):33000,가마솥 껍데기(400g):7000"
8,가람맛집,20160316,서울 노원구 동일로180길 32,한식,4.1,349,블로그 리뷰가 없습니다,
9,공릉동 소머리국밥,20211216,서울 노원구 동일로180길 53,시장,4.57,74,335,


In [39]:
insadong_df = df

In [40]:
# 534개 중 344개 크롤링 됨. (흠.......................)
# 아예 검색 결과가 없거나 주소가 완전 불일치하는 경우는 그렇다 쳐도, 종로구 내의 검색 결과는 있는데 도로명 주소가 불일치한 경우를 어떻게 처리할지,,,
insadong_df

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,이화돈까스,20190315,서울 종로구 율곡로 49-4,한식,4.36,350,78,"로스카츠:10500,히레카츠:10900,모듬카츠:12500,김치돈까스나베:12500..."
1,우미옥,19900619,서울 종로구 율곡로 230-27,한식,4.46,189,43,"설렁탕:8000,도가니탕:12000,섞어종합:35000,우족탕:12000,도가니 안..."
2,미성,19901113,서울 종로구 율곡로6길 22-1,한정식,4.04,50,2,"보리굴비정식:22000,닭볶음탕:9000,북어국:9000"
3,블루몬테,20020315,서울 종로구 율곡로 75 지하1층,양식,4.24,113,블로그 리뷰가 없습니다,
4,비비큐 치킨(인사점),19951214,서울 종로구 인사동3길 37,"치킨,닭강정",4.39,973,2,"황금올리브치킨:20000,황금올리브치킨 속안심:19000,황금올리브치킨 핫윙:200..."
...,...,...,...,...,...,...,...,...
339,47번가 찻집,20170824,서울 종로구 인사동길 50,차,4.17,84,146,"대추곶감팥빙수:17000,딸기바나나빙수:18000,망고블루베리빙수:18000,대추차..."
340,르꼬숑,20140806,서울 종로구 율곡로 83 3층,양식,4.33,850,764,"디너 B:185000,디너 A:135000,런치A (주말 및 공휴일 제외):7000..."
341,종로수제비,20140211,서울 종로구 율곡로4길 70,한식,4.24,585,27,"손수제비:7000,모듬전:25000,얼큰된장수제비:8000,납작군만두:4000,메밀..."
342,음식당,19800809,서울 종로구 인사동길 37-3 1층 음식당,"카페,디저트",4.54,82,46,"아이스크림 크로플:6000,마약로제떡볶이:7500,마약떡볶이:5500,딸기라떼:60..."


# 인사동 전처리

인사동 df의 경우 크롤링되지 않는 경우가 너무 많아 따로 전처리를 함.
이전의 크롤링 방법과 다르게 검색되는 가게 중 주소에 '종로구' 만 포함되어 있어도 df에 추가했음.

In [56]:
df_original.drop('Unnamed: 0', axis=1, inplace=True)

In [68]:
missing_df = pd.DataFrame(columns=['검색명', '사업장명', '인허가일자', '도로명주소'])

for idx, name in enumerate(df_original['사업장명']):
    if name not in list(insadong_df['음식점 이름']):
        missing_df = missing_df.append(df_original.loc[idx])
        
missing_df

,검색명,사업장명,인허가일자,도로명주소
0,시민식당,시민식당 2호점,20020206,종로11길
12,풍경,풍경,19951108,인사동7길
13,아리랑가든,아리랑가든,19860826,인사동길
18,잔치집,잔치집,20011119,인사동12길
21,예술국수,예술국수,19921230,인사동5길
...,...,...,...,...
514,코트키친,코트키친 바이 로컬릿,20210322,인사동1길
515,암돼지,암돼지 오양식관,20181127,종로11길
516,오상궁죽앤비빔밥,오상궁죽앤비빔밥,20181107,인사동길
520,실버레인,실버레인(Silver Rain) 로스터리카페&라운지,20181016,인사동길


In [81]:
missing_df

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
URL = 'https://map.naver.com/v5/?c=14122775.1818058,4513294.7120075,15,0,0,0,dh'
browser.get(URL)

c = ['음식점 이름','인허가일자','주소', '분류','평점', '방문자 리뷰 수', '블로그 리뷰 수', '메뉴']
# df = pd.DataFrame(columns = c)
n = 30
m = 5

for i in tqdm(missing_df.index):
    real_name=missing_df.loc[i]['사업장명']
    name=missing_df.loc[i]['검색명']
    loc=missing_df.loc[i]['도로명주소']
    start_date=missing_df.loc[i]['인허가일자']
    
    menu_list = []
    menu_text = ""
    
    search_box = WebDriverWait(browser, n).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'input_search'))
    )
    search_box.send_keys(name)
    
    try:
        time.sleep(2)
        pages = WebDriverWait(browser, m).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list'))
        )
        if len(pages) == 1:
            address_list = WebDriverWait(browser, m).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list > ul > li > a > div > div.place_text_box.sub.ng-star-inserted > span'))
            )
            for idx, ad in zip(range(len(address_list)), address_list):
                if '종로구' in ad.text:
                    WebDriverWait(browser, m).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, f'#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container > div > nm-list > ul > li:nth-child({idx+1}) > a > div > div:nth-child(1) > span.place_text'))
                    ).click()
                    break
        elif len(pages) == 2:
            address_list = WebDriverWait(browser, m).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container:nth-child(2) > div > nm-list > ul > li > a > div > div.place_text_box.sub.ng-star-inserted > span'))
            )
            for idx, ad in zip(range(len(address_list)), address_list):
                if '종로구' in ad.text:
                    WebDriverWait(browser, m).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, f'#container > shrinkable-layout > div > app-base > search-input-box > div > div.result_box.ng-star-inserted > div > div > instant-search-list > div > div > nm-list-container:nth-child(2) > div > nm-list > ul > li:nth-child({idx+1}) > a > div > div:nth-child(1) > span.place_text'))
                    ).click()
                    break

        browser.switch_to.default_content()
        frame = WebDriverWait(browser, m).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#entryIframe'))
                )
        browser.switch_to.frame(frame)
        
    except Exception as e:
        rate = 0
        num_of_visitor_reviews = 0
        num_of_blog_reviews = 0
        category = '카테고리가 없습니다'
        address = '주소가 없습니다'
        
    else:
        # 평점, 리뷰 개수 크롤링
        time.sleep(2)
        try:
            rate = WebDriverWait(browser, m).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span._1Y6hi._1A8_M > em'))
                    ).text
            
        except:
            rate = '평점이 없습니다'
            try:
                num_of_visitor_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(1) > a > em'))
                                        ).text
            except:
                num_of_visitor_reviews = '방문자 리뷰가 없습니다'
            
            try:
                num_of_blog_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(2) > a > em'))
                                        ).text
            except:
                num_of_blog_reviews = '블로그 리뷰가 없습니다'
        
        else:
            try:
                num_of_visitor_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(2) > a > em'))
                                        ).text
                                                                                            
            except:
                num_of_visitor_reviews = '방문자 리뷰가 없습니다'
            
            try:
                num_of_blog_reviews = WebDriverWait(browser, m).until(
                                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div > span:nth-child(3) > a > em'))
                                        ).text
            except:
                num_of_blog_reviews = '블로그 리뷰가 없습니다'
            
        # 카테고리 크롤링    
        category = WebDriverWait(browser, m).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#_title > span._3ocDE'))
        ).text
        
        # 주소 크롤링
        address = WebDriverWait(browser, m).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin._18vYz > div > ul > li._1M_Iz._1aj6- > div > a > span._2yqUQ'))
        ).text
        
        # 메뉴 및 리뷰 크롤링
        titles = WebDriverWait(browser, m).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a > span'))
        )
        for title in titles:
            if title.text == '메뉴':
                # 메뉴 크롤링
                title.click()
                try:
                    menu_names = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin > div > ul > li > a > div._2CZ7z > div._25ryC > div > span._3yfZ1'))
                                )
                    menu_prices = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin > div > ul > li > a > div._2CZ7z > div._3qFuX'))
                                )
                except:
                    try:
                        menu_names = WebDriverWait(browser, m).until(
                                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#root > div.naver_order_contents > div > div > div.order_list > div.order_list_wrap.store_delivery > div > ul > li > div > a > div > div.tit'))
                                    )
                        menu_prices = WebDriverWait(browser, m).until(
                                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#root > div.naver_order_contents > div > div > div.order_list > div.order_list_wrap.store_delivery > div > ul > li > div > a > div > div.price'))
                                    )
                    except:
                        menu_names = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#content > div.place_tab_detail > div.sc_box.d_main_menu_list > ul > li > a.list_link > div.info_area > div.tit.ellp > span > div'))
                                )
                        menu_prices = WebDriverWait(browser, m).until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#content > div.place_tab_detail > div.sc_box.d_main_menu_list > ul > li > a.list_link > div.info_area > div.price'))
                                )
                for menu_name, menu_price in zip(menu_names, menu_prices):
                    mpt=menu_price.text.replace(",","")[:-1]
                    menu_list.append(menu_name.text+':'+mpt)
                    menu_text=",".join(menu_list)
                  
                break
        
        df = df.append(dict(zip(c, [real_name,start_date,address,category, rate, num_of_visitor_reviews, num_of_blog_reviews,  menu_text])), ignore_index=True)

        
        
    finally:
        time.sleep(1)
        browser.switch_to.default_content()
        WebDriverWait(browser, m).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, '#sidebar > navbar > perfect-scrollbar > div > div.ps-content > div > ul > li:nth-child(1) > a'))
                            ).click()
        print(real_name,start_date,address,category, rate, num_of_visitor_reviews, num_of_blog_reviews,  menu_text)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\thsth\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
  3%|██▎                                                                                | 1/35 [00:11<06:39, 11.74s/it]

올로(ollo) 20140513 서울 종로구 돈화문로11가길 21 맥주,호프 평점이 없습니다 7 1 


  6%|████▋                                                                              | 2/35 [00:25<07:08, 13.00s/it]

루프탑 스페이스 오 20191114 서울 종로구 돈화문로11길 37 16층 바(BAR) 평점이 없습니다 2 6 화이트 라구 파스타:25000,햄치즈 플래터:18000,매콤한 돼지볼살과 피타브레드:25000,더블피자:20000,관자구이:22000,버섯크림뇨끼:20000,부라타 샐러드:18000,문어콩피:17000,목시 시그니쳐 스테이크 버거:18000,트러플 감자튀김:15000


  9%|███████                                                                            | 3/35 [00:34<05:52, 11.03s/it]

...

오상궁죽앤비빔밥 20181107 주소가 없습니다 카테고리가 없습니다 0 0 0 


 97%|███████████████████████████████████████████████████████████████████████████████▋  | 34/35 [05:29<00:09,  9.66s/it]

실버레인(Silver Rain) 로스터리카페&라운지 20181016 주소가 없습니다 카테고리가 없습니다 0 0 0 


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [05:38<00:00,  9.66s/it]

아라가야 20211006 주소가 없습니다 카테고리가 없습니다 0 0 0 


부가적인 전처리 과정을 거쳤기 때문에, 엄밀히 따지자면 343번 인덱스 까지와 그 이후 인덱스 사이에는 구분이 필요함.

In [83]:
df

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,이화돈까스,20190315,서울 종로구 율곡로 49-4,한식,4.36,350,78,"로스카츠:10500,히레카츠:10900,모듬카츠:12500,김치돈까스나베:12500..."
1,우미옥,19900619,서울 종로구 율곡로 230-27,한식,4.46,189,43,"설렁탕:8000,도가니탕:12000,섞어종합:35000,우족탕:12000,도가니 안..."
2,미성,19901113,서울 종로구 율곡로6길 22-1,한정식,4.04,50,2,"보리굴비정식:22000,닭볶음탕:9000,북어국:9000"
3,블루몬테,20020315,서울 종로구 율곡로 75 지하1층,양식,4.24,113,블로그 리뷰가 없습니다,
4,비비큐 치킨(인사점),19951214,서울 종로구 인사동3길 37,"치킨,닭강정",4.39,973,2,"황금올리브치킨:20000,황금올리브치킨 속안심:19000,황금올리브치킨 핫윙:200..."
...,...,...,...,...,...,...,...,...
425,샬롬(SHALOM),20200806,서울 종로구 종로 173,결혼예물,평점이 없습니다,4,블로그 리뷰가 없습니다,
426,텐팔(Ten8),20210712,서울 종로구 종로1길 50 더케이트윈타워 B동 14층,종합가전,4.37,195,블로그 리뷰가 없습니다,
427,종로칼국수,20210621,서울 종로구 돈화문로11다길 14-2,"칼국수,만두",4.29,"1,356",467,"칼국수:7500,손만두:8500,만두국:8500,칼만두국:8500,손만두 반접시 추..."
428,단아,20110412,서울 종로구 평창12길 3 B1F 2호,"사진,스튜디오",평점이 없습니다,47,블로그 리뷰가 없습니다,


최후 df 처리 - 종로구만 들어있는 가게는 음식점이 아닌 경우도 많기 때문에 이러한 경우는 제외함.

In [85]:
df['분류'].unique()

array(['한식', '한정식', '양식', '치킨,닭강정', '요리주점', '소고기구이', '종합분식', '기타숙박업',
       '이자카야', '술집', '차', '카페,디저트', '맥주,호프', '중식당', '생선회', '카페', '돼지고기구이',
       '닭갈비', '돈가스', '육류,고기요리', '곰탕,설렁탕', '만두', '양꼬치', '죽', '냉면', '국수',
       '일식당', '퓨전음식', '식료품', '찌개,전골', '보세의류', '해물,생선요리', '족발,보쌈', '포장마차',
       '오리요리', '추어탕', '해장국', '도자기', '기업', '백숙,삼계탕', '전통,민속주점', '배달대행',
       '베트남음식', '인도음식', '분식', '백반,가정식', '초밥,롤', '칼국수,만두', '이탈리아음식', '브런치',
       '예식장', '사찰음식', '미술관', '김밥', '호텔', '라면', '순대,순댓국', '주꾸미요리', '두부요리',
       '생선구이', '호떡', '막국수', '바닷가재요리', '쌈밥', '피자', '갤러리카페', '101번지남산돈까스',
       '곱창,막창,양', '꽃집,꽃배달', '닭요리', '아이스크림', '떡카페', '일본식라면', '1성급',
       '일식튀김,꼬치', '플라워카페', '채식,샐러드뷔페', '한식뷔페', '샌드위치', '감자탕', '사주카페',
       '도시락,컵밥', '패션잡화', '베이커리', '태국음식', '복합문화공간', '국밥', '테이크아웃커피', '2성급',
       '셀프,대여스튜디오', '떡볶이', '바(BAR)', '와인', '아시아음식', '매운탕,해물탕', '전통숙소',
       '방면정보', '세무,회계', '서적출판', '도시가스', '갤러리,화랑', '치과', '전,빈대떡', '라이브카페',
       '휴대폰수리', '장애인복지시설', '원단,섬유,직물', '다이어트,샐러드', '옥외,전시광고

In [123]:
df[df['분류'] == '사찰음식']

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
140,산촌,19930804,서울 종로구 인사동길 30-13,사찰음식,4.31,187,104,"점심:33000,저녁:45000,쌍화차:6000,생강차:7000,유자차:7000,보..."


In [108]:
category_list = list(df['분류'].unique())
not_category = ['기업', '꽃집,꽃배달', '패션잡화', '전통숙소', '전시,행사대행', '세무,회계', '도시가스', '방면정보', '복합문화공간'
                '결혼예물', '액자,표구', '옥외,전시광고', '여행사', '금제품전문', '종합가전', '세탁소', '사진,스튜디오',
                '플라워카페', '장애인복지시설', '원단,섬유,직물', '기업,빌딩', '공연,연극시설', '보세의류', '미술관', '갤러리,화랑',
               '휴대폰수리', '치과', '서적출판', '지역명소', '기타숙박업', '결혼예물']
category_list = [c for c in category_list if c not in not_category]
category_list

['한식', '한정식', '양식', '치킨,닭강정', '요리주점', '소고기구이', '종합분식', '이자카야', '술집', '차', '카페,디저트', '맥주,호프', '중식당', '생선회', '카페', '돼지고기구이', '닭갈비', '돈가스', '육류,고기요리', '곰탕,설렁탕', '만두', '양꼬치', '죽', '냉면', '국수', '일식당', '퓨전음식', '식료품', '찌개,전골', '해물,생선요리', '족발,보쌈', '포장마차', '오리요리', '추어탕', '해장국', '도자기', '백숙,삼계탕', '전통,민속주점', '배달대행', '베트남음식', '인도음식', '분식', '백반,가정식', '초밥,롤', '칼국수,만두', '이탈리아음식', '브런치', '예식장', '사찰음식', '김밥', '호텔', '라면', '순대,순댓국', '주꾸미요리', '두부요리', '생선구이', '호떡', '막국수', '바닷가재요리', '쌈밥', '피자', '갤러리카페', '101번지남산돈까스', '곱창,막창,양', '닭요리', '아이스크림', '떡카페', '일본식라면', '1성급', '일식튀김,꼬치', '채식,샐러드뷔페', '한식뷔페', '샌드위치', '감자탕', '사주카페', '도시락,컵밥', '베이커리', '태국음식', '복합문화공간', '국밥', '테이크아웃커피', '2성급', '셀프,대여스튜디오', '떡볶이', '바(BAR)', '와인', '아시아음식', '매운탕,해물탕', '전,빈대떡', '라이브카페', '다이어트,샐러드']

In [109]:
df_processed = pd.DataFrame(columns=c)
for idx, category in enumerate(df['분류']):
    if df.loc[idx, '분류'] in category_list:
        df_processed = df_processed.append(df.loc[idx])
        
df_processed

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,이화돈까스,20190315,서울 종로구 율곡로 49-4,한식,4.36,350,78,"로스카츠:10500,히레카츠:10900,모듬카츠:12500,김치돈까스나베:12500..."
1,우미옥,19900619,서울 종로구 율곡로 230-27,한식,4.46,189,43,"설렁탕:8000,도가니탕:12000,섞어종합:35000,우족탕:12000,도가니 안..."
2,미성,19901113,서울 종로구 율곡로6길 22-1,한정식,4.04,50,2,"보리굴비정식:22000,닭볶음탕:9000,북어국:9000"
3,블루몬테,20020315,서울 종로구 율곡로 75 지하1층,양식,4.24,113,블로그 리뷰가 없습니다,
4,비비큐 치킨(인사점),19951214,서울 종로구 인사동3길 37,"치킨,닭강정",4.39,973,2,"황금올리브치킨:20000,황금올리브치킨 속안심:19000,황금올리브치킨 핫윙:200..."
...,...,...,...,...,...,...,...,...
419,크레이지 후라이 안녕인사동점,20191008,서울 종로구 종로 33 그랑서울타워 지하1층 (115호),분식,4.35,555,163,"황제 김말이 즉석 떡볶이 (2인~3인):21000,불타는 통닭 즉석 떡볶이:2600..."
420,지화자 좋다,19651110,서울 종로구 자하문로 125,한정식,4.73,69,35,"풀(Full) 코스:120000~18000,정찬(Set) 코스:80000,비건코스(..."
421,늘솜,20180529,서울 종로구 사직로 130,"찌개,전골",4.15,48,2,"김치찌개:7000,된장찌개:7000,육개장:7000,수제비:7000,제육볶음:700..."
424,소설,19990524,서울 종로구 인사동4길 12,"맥주,호프",평점이 없습니다,5,1,맥주:15000


In [110]:
df_processed.to_csv('./data/'+street_name+'크롤링결과.csv')

In [112]:
insadong_df = df_processed 

## 각각의 df는 snu_road_df , dkb_market_df , insadong_df 에 저장되어 있음!!!

부가적인 분류 전처리 - 샤로수길

In [145]:
snu_road_df[snu_road_df['분류'] == '태닝']

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
33,몰디브태닝 서울대샤로수길점,20220104,서울 관악구 관악로14길 84 2층,태닝,4.97,125,111,


In [136]:
snu_road_df['분류'].unique()

array(['카페,디저트', '베트남음식', '와인', '요리주점', '이자카야', '덮밥', '곱창,막창,양', '바(BAR)',
       '돼지고기구이', '다이어트,샐러드', '한식', '태국음식', '일본식라면', '일식당', '양식', '맥주,호프',
       '멕시코,남미음식', '양꼬치', '일점사', '태닝', '햄버거', '주꾸미요리', '포장마차', '족발,보쌈',
       '닭갈비', '정육식당', '소고기구이', '일식튀김,꼬치', '칼국수,만두', '이탈리아음식', '닭요리',
       '인도음식', '생선회', '북카페', '육류,고기요리', '양갈비', '중식당', '브런치', '전통,민속주점',
       '스파게티,파스타전문', '술집', '치킨,닭강정', '스테이크,립', '카레', '돈가스', '오뎅,꼬치',
       '닭볶음탕', '아시아음식', '생선구이', '카페', '분식', '조개요리', '전,빈대떡', '닭발', '오리요리',
       '찜닭', '스페인음식', '종합분식', '피자', '프랑스음식', '민혁이네외국포차', '잔잔', '보드카페',
       '복합문화공간', '빙수', '우동,소바', '떡볶이'], dtype=object)

In [146]:
# 분류 변경
snu_road_df.loc[snu_road_df['분류'] == '민혁이네외국포차', '분류'] = '포장마차'
snu_road_df.loc[snu_road_df['분류'] == '잔잔', '분류'] = '이자카야'
snu_road_df.loc[snu_road_df['분류'] == '일점사', '분류'] = '육류,고기요리'

# 삭제
snu_road_df.drop(33, inplace=True) # 몰디브태닝

In [148]:
snu_road_df['분류'].unique()

array(['카페,디저트', '베트남음식', '와인', '요리주점', '이자카야', '덮밥', '곱창,막창,양', '바(BAR)',
       '돼지고기구이', '다이어트,샐러드', '한식', '태국음식', '일본식라면', '일식당', '양식', '맥주,호프',
       '멕시코,남미음식', '양꼬치', '육류,고기요리', '햄버거', '주꾸미요리', '포장마차', '족발,보쌈',
       '닭갈비', '정육식당', '소고기구이', '일식튀김,꼬치', '칼국수,만두', '이탈리아음식', '닭요리',
       '인도음식', '생선회', '북카페', '양갈비', '중식당', '브런치', '전통,민속주점', '스파게티,파스타전문',
       '술집', '치킨,닭강정', '스테이크,립', '카레', '돈가스', '오뎅,꼬치', '닭볶음탕', '아시아음식',
       '생선구이', '카페', '분식', '조개요리', '전,빈대떡', '닭발', '오리요리', '찜닭', '스페인음식',
       '종합분식', '피자', '프랑스음식', '보드카페', '복합문화공간', '빙수', '우동,소바', '떡볶이'],
      dtype=object)

부가적인 분류 전처리 - 인사동

In [176]:
insadong_df[insadong_df['분류'] == '차']

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
12,바람부는섬,19890608,서울 종로구 인사동길 20,차,4.31,78,7,"대추차:5000,동동주:6000,파전:10000,김치찌개:10000"
16,전통다원,19971222,서울 종로구 인사동10길 11-4 경인미술관,차,4.4,655,134,"대추차:7000,쌍화차:8000,생강차:7000,오미자차:7000,말차:7000,말..."
30,그 옛찻집,19950418,서울 종로구 인사동길 33-1,차,4.35,30,78,"옛날팥빙수:9000,콩가루아이스크림:9000,쫄깃 가래떡구이(4개):6000,가래떡..."
39,인사동풍경이 있는 찻집,20020325,서울 종로구 인사동길 29,차,4.28,317,53,"대추차:7500,쌍화차:7500,오미자차:7500,생강차:7500,계피차:7500,..."
48,오가다 종로2가 인사동점,19860729,서울 종로구 인사동길 60 크라운빌딩 2층,차,4.48,171,9,"초코빙수:12000,설향딸기빙수:12000,인절미오곡팥빙수:12000,애플망고빙수:..."
57,지대방,19870415,서울 종로구 인사동길 33,차,4.41,142,116,"매화차:6500,생강나무꽃차:6500,산국화차:6500,봄쑥차:7000,감잎차:70..."
96,오가다,19970411,서울 종로구 인사동길 60 크라운빌딩 2층,차,4.48,171,9,"초코빙수:12000,설향딸기빙수:12000,인절미오곡팥빙수:12000,애플망고빙수:..."
226,이원,20000127,서울 종로구 종로11길 29,차,4.37,67,6,
230,나무새,20171123,서울 종로구 인사동길 47-9,차,4.37,449,63,"특제쌍화차:9000,단팥죽:10000,제주하귤에이드:7500,메리골드:7000,아메..."
234,인사마루,20140328,서울 종로구 인사동길 35-3,차,4.26,33,49,"대추차:7500,오미자차:7500,생강차:7500,계피차:7500,유자차:7500,..."


In [152]:
insadong_df['분류'].unique()

array(['한식', '한정식', '양식', '치킨,닭강정', '요리주점', '소고기구이', '종합분식', '이자카야', '술집',
       '차', '카페,디저트', '맥주,호프', '중식당', '생선회', '카페', '돼지고기구이', '닭갈비', '돈가스',
       '육류,고기요리', '곰탕,설렁탕', '만두', '양꼬치', '죽', '냉면', '국수', '일식당', '퓨전음식',
       '식료품', '찌개,전골', '해물,생선요리', '족발,보쌈', '포장마차', '오리요리', '추어탕', '해장국',
       '도자기', '백숙,삼계탕', '전통,민속주점', '배달대행', '베트남음식', '인도음식', '분식',
       '백반,가정식', '초밥,롤', '칼국수,만두', '이탈리아음식', '브런치', '예식장', '사찰음식', '김밥',
       '호텔', '라면', '순대,순댓국', '주꾸미요리', '두부요리', '생선구이', '호떡', '막국수',
       '바닷가재요리', '쌈밥', '피자', '갤러리카페', '101번지남산돈까스', '곱창,막창,양', '닭요리',
       '아이스크림', '떡카페', '일본식라면', '1성급', '일식튀김,꼬치', '채식,샐러드뷔페', '한식뷔페',
       '샌드위치', '감자탕', '사주카페', '도시락,컵밥', '베이커리', '태국음식', '복합문화공간', '국밥',
       '테이크아웃커피', '2성급', '셀프,대여스튜디오', '떡볶이', '바(BAR)', '와인', '아시아음식',
       '매운탕,해물탕', '전,빈대떡', '라이브카페', '다이어트,샐러드'], dtype=object)

In [177]:
# 분류변경
insadong_df.loc[insadong_df['분류'] == '101번지남산돈까스', '분류'] = '돈가스'
insadong_df.loc[insadong_df['분류'] == '1성급', '분류'] = '호텔'
insadong_df.loc[insadong_df['분류'] == '2성급', '분류'] = '호텔'
insadong_df.loc[insadong_df['분류'] == '도자기', '분류'] = '차'


# 삭제
insadong_df.drop(139, inplace=True) # 루드블랑(예식장)
insadong_df.drop(95, inplace=True) # 친구(배달대행)
insadong_df.drop(366, inplace=True) # 인사동쌈지(복합문화공간)
insadong_df.drop(265, inplace=True) # 인사동[닭갈비~](복합문화공간)


In [178]:
insadong_df['분류'].unique()

array(['한식', '한정식', '양식', '치킨,닭강정', '요리주점', '소고기구이', '종합분식', '이자카야', '술집',
       '차', '카페,디저트', '맥주,호프', '중식당', '생선회', '카페', '돼지고기구이', '닭갈비', '돈가스',
       '육류,고기요리', '곰탕,설렁탕', '만두', '양꼬치', '죽', '냉면', '국수', '일식당', '퓨전음식',
       '식료품', '찌개,전골', '해물,생선요리', '족발,보쌈', '포장마차', '오리요리', '추어탕', '해장국',
       '백숙,삼계탕', '전통,민속주점', '베트남음식', '인도음식', '분식', '백반,가정식', '초밥,롤',
       '칼국수,만두', '이탈리아음식', '브런치', '사찰음식', '김밥', '호텔', '라면', '순대,순댓국',
       '주꾸미요리', '두부요리', '생선구이', '호떡', '막국수', '바닷가재요리', '쌈밥', '피자',
       '갤러리카페', '곱창,막창,양', '닭요리', '아이스크림', '떡카페', '일본식라면', '일식튀김,꼬치',
       '채식,샐러드뷔페', '한식뷔페', '샌드위치', '감자탕', '사주카페', '도시락,컵밥', '베이커리',
       '태국음식', '국밥', '테이크아웃커피', '셀프,대여스튜디오', '떡볶이', '바(BAR)', '와인',
       '아시아음식', '매운탕,해물탕', '전,빈대떡', '라이브카페', '다이어트,샐러드'], dtype=object)

In [179]:
snu_road_df.to_csv('./data/'+'샤로수길'+'크롤링결과.csv')

In [180]:
insadong_df.to_csv('./data/'+'인사동'+'크롤링결과.csv')

## 연남동, 서래마을, 남대문시장 분류 처리

In [250]:
yeonnam_df = pd.read_csv('./data/연남동크롤링결과.csv', sep='\t', index_col=0)
seorae_df = pd.read_csv('./data/서래마을카페거리크롤링결과.csv', sep='\t', index_col=0)
namdaemoon_df = pd.read_csv('./data/남대문시장크롤링결과.csv', sep='\t', index_col=0)

분류 전처리 - 연남동

In [228]:
yeonnam_df[yeonnam_df['분류'] == '서점']

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
33,서점리스본 포르투,20190617,"서울 마포구 성미산로23길 60 1~3층, 경의선 숲길에 있는 동네서점입니다., 경...",서점,4.56,46,135,NaN
249,책크인,20200716,서울 마포구 성미산로29안길 29 2층,서점,4.82,19,50,NaN
385,북스피리언스(booksperience),20160310,서울 마포구 연남로11길 34 B1,서점,평점이 없습니다,방문자 리뷰가 없습니다,블로그 리뷰가 없습니다,NaN


In [257]:
yeonnam_df[yeonnam_df['음식점 이름'] == '북스피리언스(booksperience)']

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
372,북스피리언스(booksperience),20160310,서울 마포구 연남로11길 34 B1,와인,평점이 없습니다,방문자 리뷰가 없습니다,블로그 리뷰가 없습니다,NaN


In [209]:
yeonnam_df['분류'].unique()

array(['양식', '요리주점', '도넛', '소고기구이', '바(BAR)', '중식당', '카페,디저트', '해물,생선요리',
       '이자카야', '한식', '돈가스', '아시아음식', '치킨,닭강정', '샌드위치', '육류,고기요리', '찜닭',
       '생선회', '카페', '이탈리아음식', '분식', '일식당', '스파게티,파스타전문', '서점', '전통,민속주점',
       '퓨전음식', '독립서점', '멕시코,남미음식', '와인', '일본식라면', '햄버거', '피자', '곱창,막창,양',
       '임대업', '맥주,호프', '떡볶이', '주택', '브런치', '주류', '김밥', '서적출판', '덮밥',
       '아귀찜,해물찜', '생선구이', '국수', '양꼬치', '다이어트,샐러드', '핫도그', '종합분식', '포장마차',
       '감자탕', '공방', '족발,보쌈', '자매떡볶이', '기업', '보드카페', '임대,대여', '베이커리',
       '사진,스튜디오', '기사식당', '닭볶음탕', '아이스크림', '스터디카페', '술집', '베트남음식',
       '인테리어소품', '오므라이스', '일식튀김,꼬치', '스페인음식', '냉면', '케이크전문', '초밥,롤', '차',
       '쌈밥', '한정식', '행정복지센터', '카레', '닭발', '테이크아웃커피', '낙지요리', '라이브카페',
       '정육식당', '광고,마케팅', '파출소', '2성급', '해장국', '여성의류', '돼지고기구이', '태국음식',
       '샤브샤브', '양갈비', '비빔밥', '백반,가정식', '찌개,전골', '프랑스음식', '오리요리', '한식뷔페',
       '막국수', '오뎅,꼬치', '우동,소바', '지진대피장소', '순대,순댓국', '장어,먹장어요리', '만두',
       '주꾸미요리', '국밥', '음반기획,제작', '공연,연극시설', '대안학교', '전,빈대떡', '복어요리',

In [251]:
not_category = ['기업', '임대,대여', '사진,스튜디오', '인테리어소품', '행정복지센터',
               '광고,마케팅', '파출소', '2성급', '여성의류', '지진대피장소', '음반기획,제작',
                '공연,연극시설', '대안학교', '애견용품', '게스트하우스', '남성전문미용실',
                '미용실', '촬영장비', '반찬가게', '갤러리,화랑']
for idx, category in enumerate(yeonnam_df['분류']):
    if category in not_category:
        yeonnam_df.drop(idx, inplace=True)

In [252]:
# 분류 변경
yeonnam_df.loc[yeonnam_df['분류'] == '서점', '분류'] = '와인'
yeonnam_df.loc[yeonnam_df['음식점 이름'] == '샹볼 뮈지니', '분류'] = '와인'
yeonnam_df.loc[yeonnam_df['음식점 이름'] == '곰팡이마트', '분류'] = '와인'
yeonnam_df.loc[yeonnam_df['음식점 이름'] == '우리술사자', '분류'] = '술집'
yeonnam_df.loc[yeonnam_df['음식점 이름'] == '레더쿡', '분류'] = '와인'
yeonnam_df.loc[yeonnam_df['음식점 이름'] == '플레이그라운드(play ground)', '분류'] = '술집'
yeonnam_df.loc[yeonnam_df['음식점 이름'] == '서울콜렉터', '분류'] = '카페'



# 삭제
yeonnam_df.drop(33, inplace=True) # 서점리스본 (서점)
yeonnam_df.drop(45, inplace=True)
yeonnam_df.drop(55, inplace=True)
yeonnam_df.drop(327, inplace=True)
yeonnam_df.drop(68, inplace=True) # 아띠 (주택)
yeonnam_df.drop(95, inplace=True) # 오픈 엔디드 (서적출판)
yeonnam_df.drop(526, inplace=True) # 혜원출판사 (서적출판)
yeonnam_df.drop(182, inplace=True) # 마닐다과 (공방)
yeonnam_df.drop(681, inplace=True) # 그랑 (공방)
yeonnam_df.drop(883, inplace=True) # 플레이팅쿠킹스튜디오 (공방)




# 추가
yeonnam_df = yeonnam_df.append(pd.Series(['더배드저널', '20181115', '서울 마포구 동교로25길 57 1층', 
                               '맥주,호프', '4.83', '31', '18', '와인:38000'], index=yeonnam_df.columns), ignore_index=True)
yeonnam_df = yeonnam_df.append(pd.Series(['오픈 엔디드', '20220205', '서울 마포구 동교로 253 4층', 
                               '와인', '평점이 없습니다', '1', '5', 
                                '마늘 순대 튀김:11900,메롱메롱:변동,방울방울 보코치니 샐러드:12900,가지떡갈비구이:12000,쨔파게티:8800,그라스와인:11000'],
                                        index=yeonnam_df.columns), ignore_index=True)
yeonnam_df = yeonnam_df.append(pd.Series(['성미산알루', '20020821', '서울 마포구 성미산로1길 29 1층', 
                               '한식', '4.6', '34', '8', 
                                '가자미간장조림:24000,해물누룽지탕:22000,매운토마토홍합탕:18000,부추순대와겉절이:18000,비건함박스테이크:17000,문어오일파스타:14000'], 
                                        index=yeonnam_df.columns), ignore_index=True)

In [253]:
yeonnam_df['분류'].unique()

array(['양식', '요리주점', '도넛', '소고기구이', '바(BAR)', '중식당', '카페,디저트', '해물,생선요리',
       '이자카야', '한식', '돈가스', '아시아음식', '치킨,닭강정', '샌드위치', '육류,고기요리', '찜닭',
       '생선회', '카페', '이탈리아음식', '분식', '일식당', '스파게티,파스타전문', '전통,민속주점',
       '퓨전음식', '멕시코,남미음식', '와인', '일본식라면', '햄버거', '피자', '곱창,막창,양', '맥주,호프',
       '떡볶이', '브런치', '김밥', '덮밥', '아귀찜,해물찜', '생선구이', '국수', '양꼬치',
       '다이어트,샐러드', '핫도그', '종합분식', '포장마차', '감자탕', '족발,보쌈', '자매떡볶이', '보드카페',
       '베이커리', '기사식당', '닭볶음탕', '아이스크림', '스터디카페', '술집', '베트남음식', '오므라이스',
       '일식튀김,꼬치', '스페인음식', '냉면', '케이크전문', '초밥,롤', '차', '쌈밥', '한정식', '카레',
       '닭발', '테이크아웃커피', '낙지요리', '라이브카페', '정육식당', '해장국', '돼지고기구이', '태국음식',
       '샤브샤브', '양갈비', '비빔밥', '백반,가정식', '찌개,전골', '프랑스음식', '오리요리', '한식뷔페',
       '막국수', '오뎅,꼬치', '우동,소바', '순대,순댓국', '장어,먹장어요리', '만두', '주꾸미요리', '국밥',
       '전,빈대떡', '복어요리', '테마카페', '백숙,삼계탕', '인도음식', '강화통통생고기', '곰탕,설렁탕',
       '칼국수,만두', '갤러리카페', '그리스음식', '닭요리'], dtype=object)

In [254]:
yeonnam_df

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,헤이죠지,20211206,서울 마포구 동교로46길 42-24 1층 헤이죠지,양식,평점이 없습니다,216,372,"하우스스테이크:29000,와플치킨:22000,맥앤치즈 잠봉 파스타:16000,잠발라..."
1,무채색,20211130,서울 마포구 동교로32길 19 3층,요리주점,평점이 없습니다,28,83,"수비드 삼겹 물수육:20000,수비드 삼겹 오븐구이:19000,봉골레오일파스타:16..."
2,(주)소금집 그로서리 연남,20210323,서울 마포구 성미산로29안길 18 201호,양식,평점이 없습니다,3,2,"잠봉뵈르:14000,잠봉 프로마쥬:14000,파스트라미 샌드위치:15000,살라미 ..."
3,카페 오로라,20111216,서울 마포구 동교로 247,도넛,4.4,"3,182","2,373","Classic:2200,Deluxe:2500,Fancy:2900,Premium:33..."
4,논골집 연희점,20190709,서울 서대문구 동교로 294 1층 논골집 연희연남점,소고기구이,4.71,16,34,"논골집 양념 갈비살 150g:18000,논골집 양념 안창살 150g:18000,꽃갈..."
...,...,...,...,...,...,...,...,...
902,카탈리스트,20220509,"서울 마포구 동교로46길 42-24 3층, BAR Catalyst",바(BAR),평점이 없습니다,0,0,Cocktails:17000
903,코코펠리,20220509,서울 마포구 성미산로 35,"카페,디저트",평점이 없습니다,방문자 리뷰가 없습니다,블로그 리뷰가 없습니다,NaN
904,더배드저널,20181115,서울 마포구 동교로25길 57 1층,"맥주,호프",4.83,31,18,와인:38000
905,오픈 엔디드,20220205,서울 마포구 동교로 253 4층,와인,평점이 없습니다,1,5,"마늘 순대 튀김:11900,메롱메롱:변동,방울방울 보코치니 샐러드:12900,가지떡..."


In [258]:
yeonnam_df.to_csv('./data/'+'연남동'+'크롤링결과.csv')

분류 전처리 - 서래마을

In [264]:
seorae_df[seorae_df['분류'] == '주택']

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
69,엔젤스 다트,20141015,서울 서초구 방배중앙로23길 12,주택,평점이 없습니다,21,53,메뉴가 없습니다


In [260]:
seorae_df['분류'].unique()

array(['생선회', '맥주,호프', '육류,고기요리', '카페', '막국수', '한식', '요리주점', '전통,민속주점',
       '이자카야', '오징어요리', '김밥', '칼국수,만두', '일식당', '도시락,컵밥', '중식당', '찌개,전골',
       '이탈리아음식', '치킨,닭강정', '종합분식', '감자탕', '돈가스', '분식', '순대,순댓국', '백숙,삼계탕',
       '와인', '곱창,막창,양', '돼지고기구이', '죽', '양식', '양꼬치', '바(BAR)', '국수',
       '포장마차', '해장국', '술집', '떡,한과', '애견카페', '일본식라면', '베트남음식', '주택',
       '장어,먹장어요리', '주상복합', '태국음식', '카페,디저트', '피자', '우동,소바'], dtype=object)

In [265]:
not_category = ['주택', '주상복합', '애견카페']
for idx, category in enumerate(seorae_df['분류']):
    if category in not_category:
        seorae_df.drop(idx, inplace=True)
        
# 추가
seorae_df = seorae_df.append(pd.Series(['엔젤스다트', '20141015', '서울 서초구 방배중앙로25길 10 지하1층', 
                               '맥주,호프', '평점이 없습니다', '방문자 리뷰가 없습니다', '블로그 리뷰가 없습니다', 
                                        '카스생맥주:2900,버드와이져 생맥주:3900'], index=seorae_df.columns), ignore_index=True)
seorae_df = seorae_df.append(pd.Series(['방배 정육식당', '20151015', '서울 서초구 방배중앙로 188', 
                               '정육식당', '4.35', '230', '21', 
                                        '꽃등심(200g):15000,돼지특수부위(600g):16500'], index=seorae_df.columns), ignore_index=True)

In [268]:
seorae_df['분류'].unique()

array(['생선회', '맥주,호프', '육류,고기요리', '카페', '막국수', '한식', '요리주점', '전통,민속주점',
       '이자카야', '오징어요리', '김밥', '칼국수,만두', '일식당', '도시락,컵밥', '중식당', '찌개,전골',
       '이탈리아음식', '치킨,닭강정', '종합분식', '감자탕', '돈가스', '분식', '순대,순댓국', '백숙,삼계탕',
       '와인', '곱창,막창,양', '돼지고기구이', '죽', '양식', '양꼬치', '바(BAR)', '국수',
       '포장마차', '해장국', '술집', '떡,한과', '일본식라면', '베트남음식', '장어,먹장어요리', '태국음식',
       '카페,디저트', '피자', '우동,소바', '정육식당'], dtype=object)

In [269]:
seorae_df

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,미노루,20010403,서울 서초구 방배중앙로25길 38 1층,생선회,4.98,33,19,"참치회 코스 로얄:55000,참치회 코스 특선:38000"
1,사랑방,20011016,서울 서초구 방배중앙로25길 42,"맥주,호프",평점이 없습니다,5,1,NaN
2,횡성고기마을,20010412,서울 서초구 방배중앙로29길 3 1층,"육류,고기요리",4.16,55,15,"한우1++ 꽃등심:33000,한우1++ 특수부위:35000,한우1++ 갈비살:320..."
3,코코(CoCo),20010730,서울 서초구 방배중앙로 163,카페,평점이 없습니다,방문자 리뷰가 없습니다,블로그 리뷰가 없습니다,NaN
4,방배화로구이,20190319,서울 서초구 방배중앙로 154,"육류,고기요리",평점이 없습니다,30,16,"이베리코모둠 (450g):43000,소생갈비살(500g):54000,이베리코 꽃목살..."
...,...,...,...,...,...,...,...,...
101,착한쥬스,20181218,서울 서초구 방배중앙로25길 51-1,"카페,디저트",평점이 없습니다,47,382,메뉴가 없습니다
102,(주)커피빈코리아 방배카페골목점,20181123,서울 서초구 방배중앙로 187,카페,4.51,47,382,메뉴가 없습니다
103,방배불백앤쭈꾸미,20000108,서울 서초구 방배중앙로 188,한식,평점이 없습니다,방문자 리뷰가 없습니다,블로그 리뷰가 없습니다,NaN
104,엔젤스다트,20141015,서울 서초구 방배중앙로25길 10 지하1층,"맥주,호프",평점이 없습니다,방문자 리뷰가 없습니다,블로그 리뷰가 없습니다,"카스생맥주:2900,버드와이져 생맥주:3900"


In [270]:
seorae_df.to_csv('./data/'+'서래마을카페거리'+'크롤링결과.csv')

분류 전처리 - 남대문시장

In [276]:
namdaemoon_df[namdaemoon_df['분류'] == '기업']

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
56,성원코리아,20210611,서울 중구 남대문시장길 73-5,기업,평점이 없습니다,5,1,NaN


In [271]:
namdaemoon_df['분류'].unique()

array(['한식', '족발,보쌈', '해물,생선요리', '종합분식', '오리요리', '곰탕,설렁탕', '중식당', '만두',
       '닭요리', '냉면', '양식', '우동,소바', '먹자거리', '카페,디저트', '사격장', '생선회',
       '육류,고기요리', '백숙,삼계탕', '핫도그', '죽', '카페', '포장마차', '떡볶이', '소고기구이',
       '기업', '테이크아웃커피'], dtype=object)

In [277]:
# 삭제
namdaemoon_df.drop(28, inplace=True)
namdaemoon_df.drop(39, inplace=True)
namdaemoon_df.drop(56, inplace=True)

# 변경
namdaemoon_df.loc[namdaemoon_df['음식점 이름'] == '성원코리아', '분류'] = '육류,고기요리'

# 추가
namdaemoon_df = namdaemoon_df.append(pd.Series(['남대문갈치', '20180406', '서울 중구 남대문시장길 12-13', 
                               '한식', '4.31', '212', '8', 
                                                '갈치조림:18000,고등어조림:18000,모듬생선구이:9000,고등어구이:8000'], index=yeonnam_df.columns), ignore_index=True)


In [278]:
namdaemoon_df['분류'].unique()

array(['한식', '족발,보쌈', '해물,생선요리', '종합분식', '오리요리', '곰탕,설렁탕', '중식당', '만두',
       '닭요리', '냉면', '양식', '우동,소바', '카페,디저트', '생선회', '육류,고기요리', '백숙,삼계탕',
       '핫도그', '죽', '카페', '포장마차', '떡볶이', '소고기구이', '테이크아웃커피'], dtype=object)

In [279]:
namdaemoon_df

,음식점 이름,인허가일자,주소,분류,평점,방문자 리뷰 수,블로그 리뷰 수,메뉴
0,넝쿨식당,19730503,서울 중구 남대문시장길 18-10,한식,4.18,152,37,NaN
1,순천왕족발,19971107,서울 중구 남대문시장4길 12-1,"족발,보쌈",4.37,44,15,"족발:30000,숯불 소금구이:13000,순대국:6000,해장국:6000"
2,전주식당,19760423,서울 중구 남대문시장길 18-7,"해물,생선요리",4.08,201,123,"갈치조림:20000,고등어조림:20000,모듬생선구이:10000,삼치구이:10000..."
3,넥타이맨갈치동태,19770207,서울 중구 남대문시장길 18-9,"해물,생선요리",4.27,105,67,"국산갈치조림(1인):9000,힘나는 동태탕:7000,두부조림:6000,묵은지 고등어..."
4,남문교자,19750301,서울 중구 남대문시장길 37,종합분식,4.38,168,70,"칼만두:9000,칼국수:8000,왕만두:8000,만두국:8000,떡만두국:9000,..."
5,이모네,20051103,서울 중구 남대문시장길 18-11,"해물,생선요리",4.18,73,49,"김치찌개:7000,된장찌개:6000,고등어조림:8000"
6,동화식당,19970213,서울 중구 남대문시장길 18-8,한식,4.34,171,70,"갈치조림:10000,고등어조림:10000,갈치구이:10000,고등어구이:9000,청..."
7,남문토방,19970509,서울 중구 남대문시장길 45-3 부자프라자 지하1층,오리요리,4.2,방문자 리뷰가 없습니다,블로그 리뷰가 없습니다,"유황오리훈제샤브샤브(대):49000,추어탕:9000,유황오리생구이(대):45000,..."
8,진주집,19941021,서울 중구 남대문시장길 22-2 .,"곰탕,설렁탕",4.32,131,348,"꼬리토막:29000,해장국:9000,꼬리찜:80000,설렁탕:10000,도가니탕:1..."
9,춘향식당,20120910,서울 중구 남대문시장길 16-18,한식,4.46,24,16,"닭볶음탕:26000,갈치조림 2인:20000,고등어자반구이:9000,제육볶음 2인:..."


In [282]:
namdaemoon_df.to_csv('./data/'+'남대문시장'+'크롤링결과.csv', sep='\t')

### 이후 처리 test용 

In [134]:
for name, menus in zip(df['음식점 이름'], df['메뉴']):
    if menus == []:
        print(name)

해원
커피볶는女子
반지하블루스
주식회사
몰디브태닝
종로풍

육서담
옥분이
셰프박명주브라더
도운
돼지마음포차
피엠푸드
모베터
스탑싸인
로향샤브샤브&마라탕
샤로수길
금수사
낙성대골목길
순이네밥상
양꼬치&마라탕제작소
곱창의
광안리
트라토리아체나
와룸보드게임카페
미플레이스
차이트 커피스탠드
말숙
미원포차
가득
듀레스트


In [148]:
for index, rate in zip(range(len(df_copy)), df_copy['평점']):
    if df.loc[index, '평점'] == '평점이 없습니다':
        df.loc[index, '평점'] = '0'

In [156]:
cnt = 0
for index, rate in zip(range(len(df)), df['평점']):
    if isinstance(rate, int):
        df.loc[index, '평점'] = '0'
        


In [157]:
sorted(df['평점'])

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '4',
 '4.02',
 '4.05',
 '4.06',
 '4.07',
 '4.1',
 '4.15',
 '4.16',
 '4.17',
 '4.17',
 '4.17',
 '4.18',
 '4.19',
 '4.19',
 '4.19',
 '4.22',
 '4.22',
 '4.24',
 '4.25',
 '4.26',
 '4.27',
 '4.27',
 '4.27',
 '4.27',
 '4.28',
 '4.28',
 '4.28',
 '4.29',
 '4.29',
 '4.29',
 '4.29',
 '4.3',
 '4.3',
 '4.31',
 '4.32',
 '4.32',
 '4.32',
 '4.32',
 '4.33',
 '4.33',
 '4.34',
 '4.34',
 '4.35',
 '4.35',
 '4.35',
 '4.35',
 '4.35',
 '4.35',
 '4.36',
 '4.36',
 '4.36',
 '4.36',
 '4.37',
 '4.37',
 '4.37',
 '4.37',
 '4.38',
 '4.38',
 '4.38',
 '4.38',
 '4.38',
 '4.38',
 '4.39',
 '4.39',
 '4.4',
 '4.4',
 '4.4',
 '4.4',
 '4.4',
 '4.42',
 '4.42',
 '4.42',
 '4.43',
 '4.43',
 '4.43',
 '4.44',
 '4.44',
 '4.44',
 '4.44',
 '4.44',
 '4.44',
 '4.45',
 '4.45',
